# chap18 시퀸스 배열로 다루는 순환 신경망 (RNN)

In [ ]:
#로이터 뉴스 데이터 로드를 함
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd
#num_words= 1000 단어의 빈도수가 1~1000 사이의 단어만 가져옴
(X_train,y_train),(X_test,y_test) = reuters.load_data(num_words=1000,test_split=0.2)

In [ ]:
print(X_train.shape,X_test.shape,X_test.shape[0]+X_train.shape[0])

In [ ]:
print(len(np.unique(y_train))) #46개의 카테고리 확인

#X_train 학습용 기사 , y_train 뉴스의 카테고리 46개 존재 
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical


X_train=sequence.pad_sequences(X_train, maxlen=100)
X_test=sequence.pad_sequences(X_test, maxlen=100)
y_train=to_categorical(y_train,46)
y_test=to_categorical(y_test,46)
# y_train_padded = pad_sequences(y_train, maxlen=100, padding='post', truncating='post')
# y_test_padded = pad_sequences(y_test, maxlen=100, padding='post', truncating='post')

In [ ]:
 from tensorflow.keras.models import Sequential
 from tensorflow.keras.layers import Dense,Embedding,LSTM
 from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
 


In [ ]:
model=Sequential()
model.add(Embedding(input_dim=1000, output_dim=100))# 1000 기사당 단어수 100
model.add(LSTM(100,activation='tanh'))#lstm으로 rnn 구현
model.add(Dense(46, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Early_Stopping=EarlyStopping(monitor='val_loss', patience=10, verbose=0)
history= model.fit(X_train, y_train, epochs=100,verbose=1, batch_size=20,validation_data=(X_test,y_test), callbacks=[Early_Stopping])


In [ ]:
model.evaluate(X_test, y_test ,verbose=1)[1]
import matplotlib.pyplot as plt
y_vloss=history.history['val_loss']
y_loss=history.history['loss']
x_len=np.arange(len(y_vloss))
plt.plot(x_len,y_loss,label='loss',marker='.',c='red')
plt.plot(x_len,y_vloss,label='v_loss',marker='.',c='blue')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
test_docs=['중부 지방은 대체로 맑으나 남부 지방은 구름이 많겠습니다',
           '이번 선거에는 누가 이길 수 있을까?',
           '올 초부터 유동성의 힘으로 주가가 일정하게 상승']
from tensorflow.keras.preprocessing.text import Tokenizer
token=Tokenizer()
token.fit_on_texts(test_docs)

test_docs_token=token.texts_to_sequences(test_docs)
[np.argmax(line) for line in model.predict(sequence.pad_sequences(test_docs_token,maxlen=100))]
# np.argmax(model.predict(sequence.pad_sequences(test_docs_token,maxlen=100)))

# LSTM 과 CNN 의 조합을 이용함 영화 리뷰 분류하기

In [28]:
# IMDB(영화리뷰 )데이터셋에서 데이터 가져옴
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, MaxPooling1D, Conv1D
from tensorflow.keras.preprocessing import sequence

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)
print(len(x_train), len(x_test))


25000 25000


In [29]:
#단어의 수를 맞춤
x_train= sequence.pad_sequences(x_train, maxlen=500)
x_test= sequence.pad_sequences(x_test, maxlen=500)

In [ ]:
#모델설정 -> dropout(0.5),conv1d(64,커널사이즈 4 추가) #임베딩 단어 압축하는거
model=Sequential()
model.add(Embedding(5000,500))
model.add(Conv1D(64,4,activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(4))
model.add(LSTM(55))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
early_Stopping=EarlyStopping(monitor='val_loss', patience=5, verbose=0)
history=model.fit(x_train,y_train,epochs=100,validation_split=0.25,callbacks=[early_Stopping],batch_size=512)


In [ ]:
model.evaluate(x_test, y_test ,verbose=1)[1]
import matplotlib.pyplot as plt
y_vloss=history.history['val_loss']
y_loss=history.history['loss']
x_len=np.arange(len(y_vloss))
plt.plot(x_len,y_loss,label='loss',marker='.',c='red')
plt.plot(x_len,y_vloss,label='v_loss',marker='.',c='blue')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model.predict(x_test,batch_size=128)

In [ ]:
plt.bar(range(len(x_test)),model.predict(x_test,batch_size=128).flatten())
plt.show()

In [ ]:
# 자연어 처리를 위한 딥러닝

##데이터 전처리 과정 ##
# 불필요한 단어 제거
# 문장을 단어로 토큰화
# 각 문장을 인덱스의 배열로 변환시킴
# 각 문장을 같은 사이즈로 패딩
#카테고리를 원-핫 인코딩 (옵션)

## 모델 설정및 실행 ##
model=Sequential()
model.add(Embedding(input_dim=1000,output_dim=100) #단어 임베딩 
model.add(Conv1D(filters=32,kernel_size=3,activation='relu')) #CNN 추가
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4)) #과적합 방지를 위한 
model.LSTM(units=128,activation='relu') #순서가있는 자료를 처리하기위함 RNN방법
model.summary()
# 모델 실행 옵션
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test),validation_split=0.25,callbacks=[])

model.evaluate(x_test,y_test,batch_size=128)
model.predict(x_test,batch_size=128)


In [30]:
from attention import Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,Conv1D,MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
model=Sequential()
model.add(Embedding(5000,500))
model.add(Conv1D(64,4,activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(4))
model.add(LSTM(55,return_sequences=True))
model.add(Attention())
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 500)         2500000   
                                                                 
 conv1d_10 (Conv1D)          (None, None, 64)          128064    
                                                                 
 dropout_10 (Dropout)        (None, None, 64)          0         
                                                                 
 max_pooling1d_9 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 lstm_9 (LSTM)               (None, None, 55)          26400     
                                                                 
 attention_2 (Attention)     (None, 128)               17105     
                                                     

In [31]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
early_Stopping=EarlyStopping(monitor='val_loss', patience=5, verbose=0)
history=model.fit(x_train,y_train,epochs=100,validation_split=0.25,callbacks=[early_Stopping],batch_size=512)

Epoch 1/100
37/37 [==============================] - 113s 3s/step - loss: 0.5405 - accuracy: 0.7236 - val_loss: 0.3476 - val_accuracy: 0.8594
Epoch 2/100
37/37 [==============================] - 99s 3s/step - loss: 0.2496 - accuracy: 0.8996 - val_loss: 0.2968 - val_accuracy: 0.8758
Epoch 3/100
37/37 [==============================] - 92s 2s/step - loss: 0.1773 - accuracy: 0.9351 - val_loss: 0.3016 - val_accuracy: 0.8728
Epoch 4/100
37/37 [==============================] - 87s 2s/step - loss: 0.1376 - accuracy: 0.9524 - val_loss: 0.2885 - val_accuracy: 0.8792
Epoch 5/100
37/37 [==============================] - 87s 2s/step - loss: 0.1038 - accuracy: 0.9637 - val_loss: 0.3274 - val_accuracy: 0.8686
Epoch 6/100
37/37 [==============================] - 91s 2s/step - loss: 0.0696 - accuracy: 0.9763 - val_loss: 0.3489 - val_accuracy: 0.8779
Epoch 7/100
37/37 [==============================] - 91s 2s/step - loss: 0.0522 - accuracy: 0.9825 - val_loss: 0.4504 - val_accuracy: 0.8675
Epoch 8/100
